# Scraper for Twitter 

Package: https://github.com/Mottl/GetOldTweets3

In [3]:
!pip install GetOldTweets3 

In [0]:
import GetOldTweets3 as got
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [5]:
import nltk
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Scraping
output: dataframe and csv of tweets for given criterias

In [0]:
topic = 'virus'
text_query = 'коронавірус'
since_date = '2020-01-01'
until_date = str(datetime.date(datetime.now()))
count = 700

tweetCriteria = got.manager.TweetCriteria().setQuerySearch(text_query).setSince(since_date).setUntil(until_date).setMaxTweets(count)# Creation of list that contains all tweets
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

In [0]:
#text of tweets to list
text_tweets = [tweet.text for tweet in tweets]

In [0]:
i = 1
for tweet in tweets:
    text_file = open('output/' + str(i) + '.txt', "w", encoding='utf8')
    n = text_file.write(tweet.text)
    text_file.close()
    i += 1

In [0]:
#initial text of the tweets to txt
with open("output/virus_initial.txt", "w") as file:
    for t in range(len(tweets)):
        file.write(str(t) +'\t'+ tweets[t].text + '\n')

In [0]:
#text of tweets to the dataframe
df = pd.DataFrame(text_tweets, columns = ['Text'])
df.to_csv(topic, sep=',')

In [0]:
for i in df:
    i.to_csv(str(i)+ '.txt',index=False, sep='\t', encoding = 'utf-8')

AttributeError: 'str' object has no attribute 'to_csv'

## Cleaning the data
output: same but without punctuation, mentions or links

In [0]:
cleaned_text = text_tweets.copy()

for i in range(len(cleaned_text)):
    cleaned_text[i] = re.sub(r'((http\S+)|(@\w+)|(#\w+)|[a-zA-Z0-9]|\]|\[|[«»!#@$%^&*()=_+-|;\':",.<>?])', '', cleaned_text[i], flags=re.MULTILINE).lower()
    cleaned_text[i] = re.sub(r'  ', ' ', cleaned_text[i]).split()
    # Additional bad symbols that can't be put in the RegEx
    cleaned_text[i] = [w.replace('—', '') for w in cleaned_text[i]]
    cleaned_text[i] = [w.replace('–', '') for w in cleaned_text[i]]

In [0]:
df = pd.DataFrame([[tweet] for tweet in cleaned_text], columns = ['Text'])
df.to_csv(topic + "_cleaned", sep=',')

## Getting rid of stop words
output: same but without stop words

In [0]:
stop_words_ru = ['вам', 'да', 'том', 'раз', 'всего', 'чтоб', 'можно', 'что','где', 'а', 'этого', 'кем', 'она', 'нему', 'тем', 'им', 'лет', 'мною', 'меня', 'сие', 'был', 'была', 'того', 'на', 'уж', 'какой', 'за', 'никогда', 'чуть', 'конечно', 'вся', 'без', 'он', 'все', 'той', 'ест', 'надо', 'как', 'но', 'одни', 'здесь', 'нельзя', 'для', 'этой', 'сейчас', 'бы', 'хоть', 'кому', 'были', 'через', 'коли', 'лучше', 'могу', 'такой', 'одна', 'ими', 'чего', 'при', 'у', 'даже', 'ведь', 'мое', 'иногда', 'тогда', 'будет', 'весь', 'наш', 'было', 'этих', 'его', 'или', 'опять', 'нею', 'нем', 'в', 'тот', 'когда', 'наконец', 'по', 'мой', 'перед', 'моя', 'чтобы', 'чей', 'до', 'одно', 'нее', 'чья', 'о', 'будто', 'два', 'вас', 'после', 'что', 'более', 'тех', 'с', 'моею', 'ему', 'ли', 'впрочем', 'ваш', 'потом', 'какая', 'другой', 'вон', 'всю', 'них', 'моем', 'три', 'это', 'ней', 'моей', 'теперь', 'нибудь', 'эту', 'мол', 'об', 'разве', 'вниз', 'куда', 'лишь', 'я', 'нас', 'тут', 'много', 'ним', 'сей', 'мне', 'сам', 'есть', 'потому', 'больше', 'над', 'себе', 'быть', 'всех', 'дай', 'мои', 'про', 'к', 'ей', 'вот', 'ето', 'ну', 'эта', 'тоже', 'их', 'они', 'со', 'если', 'вдруг', 'тебя', 'будь', 'оно', 'свою', 'руб', 'один', 'во', 'едим', 'себя', 'ваше', 'этот', 'чем', 'то', 'может', 'еще', 'уже', 'кто', 'под', 'мы', 'ее', 'только', 'нам', 'мою', 'там', 'хорошо', 'нашу', 'же', 'между', 'тыс', 'всегда', 'совсем', 'ними', 'вы', 'иже', 'из', 'зачем', 'ты', 'так', 'ж', 'почти', 'него', 'от', 'либо', 'эти', 'и', 'ничего', 'этом']
stop_words_uk = ['шо','б','тоді', 'тим','чим', 'тож', 'отак', 'із', 'бо', 'але','і','та','її','вона','ти','тобі','що', 'як','ще','в','або','а','от','у','з', 'які', 'чим', 'це', 'ці','й']
stop_words = stop_words_ru + stop_words_uk

In [0]:
for j in range(len(cleaned_text)):
    for i in ((cleaned_text[j])):
        if i in stop_words:
            cleaned_text[j].remove(i)

In [0]:
#print(cleaned_text)

## Lemmatisation
output: dataframe and csv of tweets for given criterias lemmatized

In [0]:
import pymorphy2

In [0]:
morph = pymorphy2.MorphAnalyzer()

lemmatized_text = cleaned_text.copy()
for j in range(len(lemmatized_text)):
    for i in range(len(lemmatized_text[j])):
        lemmatized_text[j][i]= morph.parse(lemmatized_text[j][i])[0].normal_form

In [0]:
temp = [[tweet] for tweet in lemmatized_text]
df = pd.DataFrame(temp, columns = ['Text'])
df.to_csv(topic + "_lemmatized", sep=',')

## Stemming
output: dataframe and csv of tweets for given criterias stemmed

In [0]:
from nltk.stem.snowball import RussianStemmer

In [0]:
stemmer = RussianStemmer(False)

stemmed_text = cleaned_text.copy()
for j in range(len(stemmed_text)):
    for i in range(len(stemmed_text[j])):
        stemmed_text[j][i]= stemmer.stem(stemmed_text[j][i])

In [0]:
temp = [[tweet] for tweet in stemmed_text]
df = pd.DataFrame(temp, columns = ['Text'])
df.to_csv(topic + "_stemmed", sep=',')

## additional stuff

In [0]:
def username_tweets_to_csv(username, count):
    tweetCriteria = got.manager.TweetCriteria().setUsername(username).setMaxTweets(count)
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    user_tweets = [[tweet.date, tweet.text, tweet.geo] for tweet in tweets]
    tweets_df = pd.DataFrame(user_tweets, columns = ['Datetime', 'Text' , 'Geo'])
    tweets_df.to_csv('{}-{}k-tweets.csv'.format(username, int(count/1000)), sep=',')

In [0]:
def text_query_to_csv(text_query, count, name):
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(text_query).setMaxTweets(count)
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    text_tweets = [[tweet.date, tweet.text] for tweet in tweets]
    tweets_df = pd.DataFrame(text_tweets, columns = ['Datetime', 'Text'])
    #tweets_df.to_csv('{}-{}k-tweets.csv'.format(text_query, int(count/1000)), sep=',')
    tweets_df.to_csv(name, sep=',')

In [0]:
# Input username(s) to scrape tweets and name csv file
username = 'tired_waffle'
count = 5000
username_tweets_to_csv(username, count)

#text_query = 'погода Київ'
#count = 100
#text_query_to_csv(text_query, count, "try_query")